In [1]:
import geopandas as gpd
import pandas as pd
import laspy

## Read data

In [2]:
path_to_data = "../data/3dm_32_280_5652_1_nw.laz"

In [3]:
with laspy.open(path_to_data) as f:
    print(f"Point format:       {f.header.point_format}")
    print(f"Number of points:   {f.header.point_count}")
    print(f"Number of vlrs:     {len(f.header.vlrs)}")
    header= f.header

Point format:       <PointFormat(1, 0 bytes of extra dims)>
Number of points:   15787378
Number of vlrs:     2


In [4]:
header.version

Version(major=1, minor=2)

In [5]:
crs_object = header.parse_crs()
crs_object

<Projected CRS: EPSG:25832>
Name: ETRS89 / UTM zone 32N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Europe between 6°E and 12°E: Austria; Belgium; Denmark - onshore and offshore; Germany - onshore and offshore; Norway including - onshore and offshore; Spain - offshore.
- bounds: (6.0, 38.76, 12.01, 84.33)
Coordinate Operation:
- name: UTM zone 32N
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989 ensemble
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [6]:
laz_data = laspy.read(path_to_data)

In [7]:
attributes = dir(laz_data.points[0])
print(attributes)

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_append_zeros_if_too_small', 'array', 'change_scaling', 'copy', 'copy_fields_from', 'empty', 'from_buffer', 'from_point_record', 'memoryview', 'offsets', 'point_format', 'point_size', 'resize', 'scales', 'sub_fields_dict', 'validate_dimension_name', 'zeros']


In [8]:
laz_data.points[43].number_of_returns

<SubFieldView(1)>

In [9]:
len(laz_data.points.X)

15787378

In [10]:
laz_data.points.intensity

array([41571, 45066, 40981, ..., 23483, 20403, 21932],
      shape=(15787378,), dtype=uint16)

## Write data to a GDF

In [11]:
lidar_points_gdf = gpd.GeoDataFrame({"intensity": laz_data.points.intensity},
                                    geometry=gpd.points_from_xy(laz_data.points.x, laz_data.points.y, crs=crs_object))

In [12]:
lidar_points_gdf.head()

,intensity,geometry
0,41571,POINT (280000 5652013.19)
1,45066,POINT (280000 5652013.48)
2,40981,POINT (280000.32 5652013.23)
3,40959,POINT (280000.59 5652013.26)
4,45284,POINT (280000.32 5652013.52)


In [13]:
lidar_points_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 15787378 entries, 0 to 15787377
Data columns (total 2 columns):
 #   Column     Dtype   
---  ------     -----   
 0   intensity  uint16  
 1   geometry   geometry
dtypes: geometry(1), uint16(1)
memory usage: 150.6 MB


## Visualize map

In [14]:
import plotly.express as px

In [15]:
lidar_points_gps_gdf = lidar_points_gdf.to_crs("EPSG:4326")

In [21]:
N = 1000

fig = px.scatter_map(lidar_points_gps_gdf.iloc[:N], 
                     lat=lidar_points_gps_gdf.iloc[:N].geometry.y, lon=lidar_points_gps_gdf.iloc[:N].geometry.x, 
                     color="intensity",
                     color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=19)
fig.show()

In [17]:
lidar_points_gps_gdf.iloc[:N]

,intensity,geometry
0,41571,POINT (5.86592 50.97762)
1,45066,POINT (5.86592 50.97763)
2,40981,POINT (5.86593 50.97762)
3,40959,POINT (5.86593 50.97762)
4,45284,POINT (5.86593 50.97763)
...,...,...
995,43973,POINT (5.866 50.97768)
996,22456,POINT (5.866 50.97768)
997,32876,POINT (5.866 50.97768)
998,38578,POINT (5.86601 50.97768)


## Get buildings using overpass

In [23]:
import requests

overpass_url = "https://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
(
  way["building"](south,west,north,east);
  relation["building"](south,west,north,east);
);
out body;
>;
out skel qt;
"""

# Replace with your actual bounding box coordinates
south, west, north, east = 48.137, 11.575, 48.140, 11.578
query = overpass_query.replace("south", str(south)).replace("west", str(west)).replace("north", str(north)).replace("east", str(east))

response = requests.post(overpass_url, data=query)
data = response.json()

# Process the data as needed
print(data)

{'version': 0.6, 'generator': 'Overpass API 0.7.62.4 2390de5a', 'osm3s': {'timestamp_osm_base': '2025-02-04T19:06:02Z', 'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'}, 'elements': [{'type': 'way', 'id': 4054817, 'nodes': [21486944, 2545258014, 2545258017, 4348232951, 2950486585, 282341977, 21486945, 8220727414, 2950486587, 2950486605, 8220727413, 21486942, 618914977, 2950486612, 4348232954, 2950486614, 21486943, 2950486613, 21486944], 'tags': {'addr:city': 'München', 'addr:country': 'DE', 'addr:housenumber': '15', 'addr:postcode': '80331', 'addr:street': 'Marienplatz', 'architect': 'Jörg von Halspach', 'architect:wikidata': 'Q123464', 'building': 'government', 'building:architecture': 'gothic', 'building:levels': '2', 'height': '25', 'heritage': '4', 'heritage:operator': 'BLfD', 'layer': '1', 'name': 'Altes Rathaus', 'name:de': 'Altes Rathaus', 'name:en': 'Old Town Hall', 'name:es': 'Viejo Ayuntamiento', 'name:ru

In [24]:
data

{'version': 0.6,
 'generator': 'Overpass API 0.7.62.4 2390de5a',
 'osm3s': {'timestamp_osm_base': '2025-02-04T19:06:02Z',
  'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
 'elements': [{'type': 'way',
   'id': 4054817,
   'nodes': [21486944,
    2545258014,
    2545258017,
    4348232951,
    2950486585,
    282341977,
    21486945,
    8220727414,
    2950486587,
    2950486605,
    8220727413,
    21486942,
    618914977,
    2950486612,
    4348232954,
    2950486614,
    21486943,
    2950486613,
    21486944],
   'tags': {'addr:city': 'München',
    'addr:country': 'DE',
    'addr:housenumber': '15',
    'addr:postcode': '80331',
    'addr:street': 'Marienplatz',
    'architect': 'Jörg von Halspach',
    'architect:wikidata': 'Q123464',
    'building': 'government',
    'building:architecture': 'gothic',
    'building:levels': '2',
    'height': '25',
    'heritage': '4',
    'heritage:operator': 'BLfD',
  

In [25]:
data.keys()

dict_keys(['version', 'generator', 'osm3s', 'elements'])